In [0]:
%pip install databricks-sdk --upgrade
dbutils.library.restartPython()

## Initialize Environment
**IMPORTANT**: Update these configuration values before running the next notebook!


In [0]:
# Database Configuration
LAKEBASE_DB_NAME = "acme-inventory-lakebase"  # Name for your Lakebase database
LAKEBASE_CAPACITY = "CU_2"

# App Configuration
APP_NAME = "acme-inventory-app"  # Name for your Databricks App
APP_DESCRIPTION = "Databricks Lakebase Inventory Management System"

# Secret Scope Configuration
SECRET_SCOPE_NAME = ""  # Name for your secret scope
SECRET_KEY_NAME = "SECRET_KEY"  # Key name for Flask secret

# Managed Postgres Catalog Configuration
CATALOG_NAME = "acme_inventory_live"  # Unity Catalog name for your Lakebase data. This will be an external catalog in UC
CREATE_CATALOG_IF_NOT_EXISTS = True  # Set to False if catalog already exists

# Analytics Catalog Configuration
ANALYTICS_CATALOG_NAME = "acme_inventory"  # Unity Catalog name for analytics catalog. Cannot be the same name as the managed postgres catalog.
ANALYTICS_SCHEMA_NAME = 'inventory_analytics'

# App Data Management
FORCE_DATA_RESET = "false"  # Set to "true" to reset/truncate data on startup.
LOAD_SAMPLE_DATA = "true"   # Set to "false" to disable sample data loading

# Dashboard Configuration
DASHBOARD_DISPLAY_NAME = "Acme Inventory Dashboard"  # Display name for the dashboard. Use a different display name to avoid duplicates with the repo template.

WAREHOUSE_ID = None  # Leave None to use default warehouse, or specify a warehouse ID

In [0]:
# Do NOT change the below config values

# Repository Configuration
REPO_URL = "https://github.com/reynoldspravindev/databricks-lakebase-inventory" # Leave this unchanged
REPO_PROVIDER = "github"  # git provider. Leave this unchanged

# Postgres Schema and Table Names
PGDATABASE = "databricks_postgres"
POSTGRES_SCHEMA = "inventory_app"
POSTGRES_TABLE = "inventory_items"
POSTGRES_CATEGORY_TABLE = "inventory_category"
POSTGRES_WAREHOUSE_TABLE = "inventory_warehouse"
POSTGRES_SUPPLIER_TABLE = "inventory_supplier"
POSTGRES_SKU_TABLE = "inventory_sku"

DASHBOARD_TEMPLATE_PATH = "dashboards/Lakebase Inventory Dashboard.lvdash.json"  # Leave this unchanged

#### Variables Initialized

In [0]:
import json
variables = [
    "LAKEBASE_DB_NAME",
    "LAKEBASE_CAPACITY",
    "APP_NAME",
    "SECRET_SCOPE_NAME",
    "SECRET_KEY_NAME",
    "CATALOG_NAME",
    "ANALYTICS_CATALOG_NAME",
    "ANALYTICS_SCHEMA_NAME",
    "CREATE_CATALOG_IF_NOT_EXISTS",
    "FORCE_DATA_RESET",
    "LOAD_SAMPLE_DATA",
    "DASHBOARD_DISPLAY_NAME",
    "REPO_URL",
    "PGDATABASE",
    "POSTGRES_SCHEMA",
    "POSTGRES_TABLE",
    "POSTGRES_CATEGORY_TABLE",
    "POSTGRES_WAREHOUSE_TABLE",
    "POSTGRES_SUPPLIER_TABLE"
]

var_dict = {var: globals().get(var, None) for var in variables}

print(json.dumps(var_dict, indent=2))

#### Setup Workspace SDK

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import apps, compute, workspace as ws_service
from databricks.sdk.service.apps import AppResource
from databricks.sdk.service.database import DatabaseCatalog
from databricks.sdk.service.apps import AppDeployment
from databricks.sdk.service.workspace import ImportFormat
from databricks.sdk.service.dashboards import Dashboard

# Initialize workspace client using notebook-native authentication
w = WorkspaceClient()

# Get current workspace info
current_user = w.current_user.me()

# Get the correct workspace URL from the Databricks context
# In a notebook, we can get the workspace URL from the notebook context
try:
    # Try to get workspace URL from dbutils (more reliable in notebooks)
    workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
    workspace_url = f"https://{workspace_url}"
except:
    # Fallback to config.host if dbutils is not available
    workspace_url = w.config.host

print(f"✓ Connected to workspace: {workspace_url}")
print(f"✓ Current user: {current_user.user_name}")
print(f"✓ User ID: {current_user.id}")